Detección de caras con webcam

In [1]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors
import imutils
import numpy as np
import cvzone
import random

1. La primera aproximación que hemos realizado es un filtro que pinta un bigote sobre la cara detectada.

In [2]:
# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Webcam connection
cap = cv2.VideoCapture(0)

bigote = cv2.imread('bigote.png', cv2.IMREAD_UNCHANGED)

#Set camera resolution
cap.set(3,640)
cap.set(4,480)

while True:
    # Get frame
    ret, frame = cap.read()
    # For HS normalization
    B, G, R = cv2.split(frame)

    # Search face with a specific setup for face and eye detection
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[1], FDet.EyeDetectors[1])
    if values is not None:
        face, eyes, shape = values

        #draws face container
        [x, y , w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            
            bigote_resized = imutils.resize(bigote, width = w // 2)

            # draws eyes and mask if available
            [lex, ley, rex, rey] = eyes

            left_eye = frame[(int)(ley) - 20:(int)(ley) + 20, (int)(lex) - 20:(int)(lex) + 20]
            right_eye = frame[(int)(rey) - 20:(int)(rey) + 20, (int)(rex) - 20:(int)(rex) + 20]            

            i = 0
            for (x, y) in shape:

                if i == 51:
                    if y - bigote_resized.shape[0] >= 0:

                        x1 = x - (bigote_resized.shape[0] // 2)
                        y1 = y - (bigote_resized.shape[1] // 2)
                        n_frame = frame[y1-20: y1 + bigote_resized.shape[1]-20, x1: x1 + bigote_resized.shape[0]]
                        
                        mask = bigote_resized[:, :, 3]
                        mask_inv = cv2.bitwise_not(mask)

                        bg_black_bigote = cv2.bitwise_and(bigote_resized, bigote_resized, mask=mask)
                        bg_black_bigote = bg_black_bigote[ : , : , 0:3]
                        bg_frame = cv2.bitwise_and(n_frame, n_frame, mask = mask_inv)

                        bigote_transparente = cv2.add(bg_black_bigote, bg_frame)

                        frame[y1-20: y1 + bigote_resized.shape[1]-20, x1: x1 + bigote_resized.shape[0]] = bigote_transparente[:, :, 0:3]

                i +=1


    cv2.imshow('Cam', frame)
    
    # Esc to finish
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break

# Close windows and release camera
cap.release()
cv2.destroyAllWindows()

2. Otra idea ha sido coger distintas partes de la cara y trasladarlas a otras posiciones.

In [3]:
# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Webcam connection
cap = cv2.VideoCapture(0)

#Set camera resolution
cap.set(3,640);
cap.set(4,480);

while True:
    # Get frame
    ret, frame = cap.read()
    # For HS normalization
    B, G, R = cv2.split(frame)

    # Search face with a specific setup for face and eye detection
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[1], FDet.EyeDetectors[1])
    if values is not None:

        face, eyes, shape = values

        #draws face container
        [x, y , w, h] = face
        if x > -1:

            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

            # draws eyes and mask if available
            [lex, ley, rex, rey] = eyes

            left_eye = frame[(int)(ley) - 20:(int)(ley) + 20, (int)(lex) - 20:(int)(lex) + 20].copy()
            right_eye = frame[(int)(rey) - 20:(int)(rey) + 20, (int)(rex) - 20:(int)(rex) + 20].copy()

            frame[(int)(ley) - 20:(int)(ley) + 20, (int)(lex) - 20:(int)(lex) + 20] = right_eye
            frame[(int)(rey) - 20:(int)(rey) + 20, (int)(rex) - 20:(int)(rex) + 20] = left_eye

            mouth = []
            mouth_position = ()
            front = []
            front_position = ()
            
            i = 0
            for (x, y) in shape:
                if i == 66:
                    mouth_position = (x, y)
                    mouth = frame[y-30:y+30, x-50:x+50].copy()
                if i == 27:
                    front_position = (x, y)
                    front = frame[y-70:y-10, x-50:x+50].copy()

                i +=1

            frame[front_position[1]-70:front_position[1]-10, front_position[0]-50:front_position[0]+50] = mouth
            frame[mouth_position[1]-30:mouth_position[1]+30, mouth_position[0]-50:mouth_position[0]+50] = front
            
    cv2.imshow('Cam', frame)
    
    # Esc to finish
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break

# Close windows and release camera
cap.release()
cv2.destroyAllWindows()

3. Por otro lado, hemos tenido la idea de dividir la cara en cuatro subframes y desordenarlos, de manera que haciendo uso del teclado podamos ordenarlos tal y como si fuera un puzzle. Con las teclas '1', '2', '3' y '4' seleccionamos uno de los frames empezando desde la esquina superior izquierda y acabando en la inferior derecha y una vez seleccionado un frame, pulsando las teclas 'a', 'w', 'd' y 's' podemos intercambiarlo de posición empezando de la misma manera desde la esquina superior izquierda hasta la inferior derecha.

In [7]:
# Face detectors interface
FDet = FaceDetectors.FaceDetector()

# Webcam connection
cap = cv2.VideoCapture(0)

main_frame = []
selected_frame = []
frames = []
index = 0

#Set camera resolution
cap.set(3,640)
cap.set(4,480)

while True:
    # Get frame
    ret, frame = cap.read()
    # For HS normalization
    B, G, R = cv2.split(frame)

    # Search face with a specific setup for face and eye detection
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[1], FDet.EyeDetectors[1])
    if values is not None:

        face, eyes, shape = values
        #draws face container
        [x, y, w, h] = face

        rectangle_width, resto_width = divmod(w, 2)
        rectangle_height, resto_height = divmod(h, 2)

        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            main_frame = frame

    cv2.imshow('Cam', frame)
    
    # Esc to finish
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break

# Close windows and release camera
cap.release()
cv2.destroyAllWindows()

frame1 = main_frame[y:y+rectangle_height, x:x+rectangle_width].copy()
frame2 = main_frame[y:y+rectangle_height, x+rectangle_width:x + w - resto_width].copy()
frame3 = main_frame[y+rectangle_height:y + h - resto_height, x:x+rectangle_width].copy()
frame4 = main_frame[y+rectangle_height:y + h - resto_height, x+rectangle_width:x + w - resto_width].copy()


if frame1.shape == frame2.shape == frame3.shape == frame4.shape:

    main_frame[y:y+rectangle_height, x:x+rectangle_width] = frame4
    frames.append(frame4)
    main_frame[y:y+rectangle_height, x+rectangle_width:x + w - resto_width] = frame3
    frames.append(frame3)
    main_frame[y+rectangle_height:y + h - resto_height, x:x+rectangle_width] = frame2
    frames.append(frame2)
    main_frame[y+rectangle_height:y + h - resto_height, x+rectangle_width:x + w - resto_width] = frame1
    frames.append(frame1)

    cv2.rectangle(main_frame, (x+rectangle_width, y+rectangle_height), (x+(rectangle_width*2), y + (rectangle_height*2)), (255, 0, 0), 2)
    selected_frame = main_frame[y:y+rectangle_height, x:x+rectangle_width].copy()
    cv2.rectangle(main_frame, (x, y), (x+rectangle_width, y + rectangle_height), (0, 0, 255), 2)
    selected_frame_position = (x, y)
    
def borrar():
    cv2.rectangle(main_frame, (x+rectangle_width, y+rectangle_height), (x+(rectangle_width*2), y + (rectangle_height*2)), (255, 0, 0), 2)
    cv2.rectangle(main_frame, (x, y), (x+rectangle_width, y + rectangle_height), (255, 0, 0), 2)
    cv2.rectangle(main_frame, (x+rectangle_width, y), (x+(rectangle_width*2), y + rectangle_height), (255, 0, 0), 2)    
    cv2.rectangle(main_frame, (x, y+rectangle_height), (x+rectangle_width, y + (rectangle_height*2)), (255, 0, 0), 2)

while True:

    key = cv2.waitKey(100)
    if key == ord('1'):
        borrar()
        selected_frame = main_frame[y:y+rectangle_height, x:x+rectangle_width].copy()
        cv2.rectangle(main_frame, (x, y), (x+rectangle_width, y + rectangle_height), (0, 0, 255), 2)
        selected_frame_position = (x, y)
    if key == ord('2'):
        borrar()
        selected_frame = main_frame[y:y+rectangle_height, x+rectangle_width:x + w - resto_width].copy()
        cv2.rectangle(main_frame, (x+rectangle_width, y), (x+(rectangle_width*2), y + rectangle_height), (0, 0, 255), 2)
        selected_frame_position = (x+rectangle_width, y)
    if key == ord('3'):
        borrar()
        selected_frame = main_frame[y+rectangle_height:y + h - resto_height, x:x+rectangle_width].copy()
        cv2.rectangle(main_frame, (x, y+rectangle_height), (x+rectangle_width, y + (rectangle_height*2)), (0, 0, 255), 2)
        selected_frame_position = (x, y+rectangle_height)
    if key == ord('4'):
        borrar()
        selected_frame = main_frame[y+rectangle_height:y + h - resto_height, x+rectangle_width:x + w - resto_width].copy()
        cv2.rectangle(main_frame, (x+rectangle_width, y+rectangle_height), (x+(rectangle_width*2), y + (rectangle_height*2)), (0, 0, 255), 2)
        selected_frame_position = (x+rectangle_width, y+rectangle_height)
        
    if key == ord('a'):
        borrar()
        temp = main_frame[y:y+rectangle_height, x:x+rectangle_width].copy()
        if temp.shape == selected_frame.shape:
            main_frame[y:y+rectangle_height, x:x+rectangle_width] = selected_frame
            main_frame[selected_frame_position[1]:selected_frame_position[1] + rectangle_height, selected_frame_position[0]:selected_frame_position[0] + rectangle_width] = temp
            selected_frame = main_frame[y:y+rectangle_height, x:x+rectangle_width].copy()
            cv2.rectangle(main_frame, (x, y), (x+rectangle_width, y + rectangle_height), (0, 0, 255), 2)
            selected_frame_position = (x, y)
    if key == ord('w'):
        borrar()
        temp = main_frame[y:y+rectangle_height, x+rectangle_width:x + w - resto_width].copy()
        if temp.shape == selected_frame.shape:
            main_frame[y:y+rectangle_height, x+rectangle_width:x + w - resto_width] = selected_frame
            main_frame[selected_frame_position[1]:selected_frame_position[1] + rectangle_height, selected_frame_position[0]:selected_frame_position[0] + rectangle_width] = temp
            selected_frame = main_frame[y:y+rectangle_height, x+rectangle_width:x + w - resto_width].copy()
            cv2.rectangle(main_frame, (x+rectangle_width, y), (x+(rectangle_width*2), y + rectangle_height), (0, 0, 255), 2)
            selected_frame_position = (x+rectangle_width, y)
    if key == ord('d'):
        borrar()
        temp = main_frame[y+rectangle_height:y + h - resto_height, x:x+rectangle_width].copy()
        if temp.shape == selected_frame.shape:
            main_frame[y+rectangle_height:y + h - resto_height, x:x+rectangle_width] = selected_frame
            main_frame[selected_frame_position[1]:selected_frame_position[1] + rectangle_height, selected_frame_position[0]:selected_frame_position[0] + rectangle_width] = temp
            selected_frame = main_frame[y+rectangle_height:y + h - resto_height, x:x+rectangle_width].copy()
            cv2.rectangle(main_frame, (x, y+rectangle_height), (x+rectangle_width, y + (rectangle_height*2)), (0, 0, 255), 2)
            selected_frame_position = (x, y+rectangle_height)
    if key == ord('s'):
        borrar()
        temp = main_frame[y+rectangle_height:y + h - resto_height, x+rectangle_width:x + w - resto_width].copy()
        if temp.shape == selected_frame.shape:
            main_frame[y+rectangle_height:y + h - resto_height, x+rectangle_width:x + w - resto_width] = selected_frame.copy()
            main_frame[selected_frame_position[1]:selected_frame_position[1] + rectangle_height, selected_frame_position[0]:selected_frame_position[0] + rectangle_width] = temp
            selected_frame = main_frame[y+rectangle_height:y + h - resto_height, x+rectangle_width:x + w - resto_width].copy()
            cv2.rectangle(main_frame, (x+rectangle_width, y+rectangle_height), (x+(rectangle_width*2), y + (rectangle_height*2)), (0, 0, 255), 2)
            selected_frame_position = (x+rectangle_width, y+rectangle_height)
        

    cv2.imshow('Puzzle', main_frame)

    # Esc to finish
    tec = cv2.waitKey(100)
    if tec & tec == 27:  # Esc
        break

# Close windows and release camera
cap.release()
cv2.destroyAllWindows()

4. Por último, hemos elaborado un filtro en el que una hamburguesa cae desde la parte superior de la pantalla en una coordenada x pseudoaleatoria haciendo uso de la librería random de python. El objetivo es comerse la hamburguesa, es decir una vez la separación entre tu labio superior y tu labio inferior sea mayor que 20, puedes comerte la hambuguesa, que una vez comida, hace respawn otra vez arriba. Además hemos añadido una puntuación para saber cuantas hamburguesas se han comido.

In [2]:
# Webcam connection
cap = cv2.VideoCapture(0)
overlay = cv2.imread('burger.png', cv2.IMREAD_UNCHANGED)
ret, frame = cap.read()
height, width, channels = frame.shape
 
burger_x = 320
burger_y = 0
 
# Face detectors interface
FDet = FaceDetectors.FaceDetector()
 
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')
 
# Face detection and eye model setup
imodoF = 1
imodoE = 1
 
debug = 0
score = 0
 
#Set camera resolution
cap.set(3,640)
cap.set(4,480)
 
while True:
 
 
    # Get frame
    t = time.time()
    ret, frame = cap.read()
 
    if burger_y == height:
        burger_y = 0
        burger_x = random.randint(40,width-40)
 
    # For HS normalization
    B, G, R = cv2.split(frame)
 
    # Search face with a specific setup for face and eye detection
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
   
    if values is not None:
        face, eyes, shape = values
 
       
 
        #draws face container
        [x, y , w, h] = face
        if x > -1:
            #cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
           
            # draws eyes and mask if available
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                # Show detected facial elements
                if imodoF > 0:
                    i = 0
 
                    upperLip = shape[62]
                    lowerLip = shape[66]
 
                    lip_separation = lowerLip[1] - upperLip[1]
 
                    if (lip_separation > 20):
 
                        # check if burger lower bound is under upper lip bound
                        if (burger_y + 90 > upperLip[1] - 10) and burger_y + 90 < (upperLip[1]-10) + 50:
                            if (burger_x > upperLip[0] -30 and burger_x < (upperLip[0]-30) + 70) or (burger_x + 100 < (upperLip[0]-30) + 70 and burger_x + 100 > (upperLip[0]-30)):
                                burger_y = 0
                                burger_x = random.randint(40,width-40)
                                score += 1
                                
                       
                    cv2.putText(frame, "Score: " + str(score), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)  # Muestra la puntuación en el frame 
 
 
    if debug:
        print("Processing time : {:.3f}".format(time.time() - t))
 
    result = cvzone.overlayPNG(frame, overlay, [burger_x,burger_y])
 
    burger_y += 6
 
    cv2.imshow('Cam', result)
   
    # Esc to finish
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break
 
 
# Close windoews and release camera
cap.release()
cv2.destroyAllWindows()

Camera 0
